In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam , schedules
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)


In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 28709 images belonging to 7 classes.


In [4]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [5]:
# Create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

# Set the learning rate schedule with ExponentialDecay
initial_learning_rate = 0.0001
decay_steps = 10000
decay_rate = 0.96
decay_learning_rate = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

# Create the Adam optimizer with the learning rate schedule
optimizer = Adam(learning_rate=decay_learning_rate)

# Compile the model with the optimizer
emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [6]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')

C:\Users\nirni\AppData\Local\Temp\ipykernel_13240\3198193097.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 280s 623ms/step - loss: 1.8093 - accuracy: 0.2539 - val_loss: 1.7369 - val_accuracy: 0.2966
Epoch 2/50
448/448 [==============================] - 275s 615ms/step - loss: 1.6469 - accuracy: 0.3588 - val_loss: 1.5746 - val_accuracy: 0.3894
Epoch 3/50
448/448 [==============================] - 280s 625ms/step - loss: 1.5492 - accuracy: 0.4021 - val_loss: 1.4889 - val_accuracy: 0.4315
Epoch 4/50
448/448 [==============================] - 259s 578ms/step - loss: 1.4778 - accuracy: 0.4327 - val_loss: 1.4134 - val_accuracy: 0.4636
Epoch 5/50
448/448 [==============================] - 260s 581ms/step - loss: 1.4092 - accuracy: 0.4598 - val_loss: 1.3546 - val_accuracy: 0.4873
Epoch 6/50
448/448 [==============================] - 244s 545ms/step - loss: 1.3546 - accuracy: 0.4849 - val_loss: 1.3219 - val_accuracy: 0.4999
Epoch 7/50
448/448 [==============================] - 237s 529ms/step - loss: 1.3086 - accuracy: 0.5062 - val_loss: 1.2901 -

Accuracy = 0.8536 